In [52]:
import pymongo
from datetime import timedelta, date
import charts

Server running in the folder /home/isaac at 127.0.0.1:40089


In [3]:
client = pymongo.MongoClient('localhost', 27017)
ganji = client['ganji']
item_sample = ganji['item_sample']

In [4]:
#只展示特定的内容：
#$slice: 分片，只展示第一个内容
# for i in item_sample.find({},{'area':{'$slice':1}, '_id':0,'price':0, 'title':0, 'pub_date':0}).limit(300):
#     print(i)
# for i in item_sample.find({'pub_date': '2015.12.31'},{'area':{'$slice':1}, '_id':0,'price':0, 'title':0}).limit(300):
#     print(i)
for i in item_sample.find({'pub_date': {'$in': ['2015.12.31', '2016.01.14']}},{'area':{'$slice':1}, '_id':0,'price':0, 'title':0}).limit(300):
    print(i)

{'pub_date': '2016.01.14', 'time': 2, 'cates': ['北京58同城', '北京二手市场', '北京二手家电', '北京二手洗衣机'], 'area': ['朝阳'], 'look': '-', 'url': 'http://bj.58.com/jiadian/24349380911041x.shtml'}
{'pub_date': '2015.12.31', 'time': 2, 'cates': ['北京58同城', '北京二手市场', '北京二手笔记本'], 'area': ['丰台'], 'look': '-', 'url': 'http://bj.58.com/bijibendiannao/24220974748090x.shtml'}
{'pub_date': '2015.12.31', 'time': 0, 'cates': ['北京58同城', '北京二手市场', '北京二手笔记本'], 'area': ['朝阳'], 'look': '-', 'url': 'http://bj.58.com/bijibendiannao/24516548452660x.shtml'}
{'pub_date': '2015.12.31', 'time': 3, 'cates': ['北京58同城', '北京二手市场', '北京二手笔记本'], 'area': ['燕郊'], 'look': '-', 'url': 'http://bj.58.com/bijibendiannao/24516186232254x.shtml'}
{'pub_date': '2016.01.14', 'time': 0, 'cates': ['北京58同城', '北京二手市场', '北京二手台式机/配件', '北京二手硬件', '北京二手电源'], 'area': ['海淀'], 'look': '9成新', 'url': 'http://bj.58.com/diannao/23598263475362x.shtml'}
{'pub_date': '2015.12.31', 'time': 2, 'cates': ['北京58同城', '北京二手市场', '北京二手笔记本'], 'area': ['通州'], 'look': '-', 'url'

In [5]:
#已经知道时间段（2015。12。27-2016。12。27）和区域名称
#求解单日内地区发帖量
#1。让程序明白时间段，获得单日日期
#2。使用find函数查找指定数据
#3。转化数据

for i in item_sample.find({},{'_id':0, 'pub_date':1}).limit(300):
    print(i)

{'pub_date': '2016.01.12'}
{'pub_date': '2016.01.14'}
{'pub_date': '2016.01.13'}
{'pub_date': '2015.12.27'}
{'pub_date': '2015.12.31'}
{'pub_date': '2016.01.12'}
{'pub_date': '2016.01.04'}
{'pub_date': '2015.12.31'}
{'pub_date': '2015.12.27'}
{'pub_date': '2015.12.29'}
{'pub_date': '2016.01.11'}
{'pub_date': '2015.12.27'}
{'pub_date': '2015.12.31'}
{'pub_date': '2016.01.04'}
{'pub_date': '2016.01.11'}
{'pub_date': '2016.01.14'}
{'pub_date': '2016.01.04'}
{'pub_date': '2015.12.31'}
{'pub_date': '2016.01.14'}
{'pub_date': '2015.12.29'}
{'pub_date': '2015.12.27'}
{'pub_date': '2015.12.31'}
{'pub_date': '2016.01.04'}
{'pub_date': '2016.01.11'}
{'pub_date': '2016.01.15'}
{'pub_date': '2015.12.29'}
{'pub_date': '2016.01.14'}
{'pub_date': '2015.12.31'}
{'pub_date': '2015.12.27'}
{'pub_date': '2016.01.04'}
{'pub_date': '2016.01.04'}
{'pub_date': '2016.01.15'}
{'pub_date': '2016.01.11'}
{'pub_date': '2015.12.29'}
{'pub_date': '2015.12.31'}
{'pub_date': '2015.12.27'}
{'pub_date': '2016.01.15'}
{

In [6]:
for i in item_sample.find():
    frags = i['pub_date'].split('-')
    if len(frags) == 1:
        date = frags[0]
    else:
        date = '{}.{}.{}'.format(frags[0], frags[1], frags[2])
    item_sample.update_one({'_id':i['_id']}, {'$set':{'pub_date':date}})

In [10]:
#让程序知道发帖时间范围：
#import date, timedelta
a = date(2015,5,10)
print(a)

2015-05-10


In [11]:
d = timedelta(days=1)
print(d)

1 day, 0:00:00


In [30]:
def get_all_dates(date1, date2):
    the_date = date(int(date1.split('.')[0]), int(date1.split('.')[1]), int(date1.split('.')[2]))
    end_date = date(int(date2.split('.')[0]), int(date2.split('.')[1]), int(date2.split('.')[2]))
    days = timedelta(days=1)
   
    while the_date <= end_date:
        yield (the_date.strftime('%Y.%m.%d'))
        the_date = the_date + days


In [31]:
for i in get_all_dates('2015.12.24', '2016.01.05'):
    print(i)

2015.12.24
2015.12.25
2015.12.26
2015.12.27
2015.12.28
2015.12.29
2015.12.30
2015.12.31
2016.01.01
2016.01.02
2016.01.03
2016.01.04
2016.01.05


In [48]:
#调整数据格式
def get_data_within(date1, date2, areas):
    for area in areas:
        area_day_posts = []
        for date in get_all_dates(date1, date2):
            a = list(item_sample.find({'pub_date':date, 'area':area}))
            each_day_post = len(a)
            area_day_posts.append(each_day_post)
        data = {
            'name': area,
            'data': area_day_posts,
            'type': 'line'
        }
        yield data

In [49]:
for i in get_data_within('2015.12.24', '2016.01.05', ['朝阳', '海淀', '通州']):
    print(i)

{'data': [220, 217, 259, 266, 322, 287, 309, 307, 346, 440, 488, 641, 649], 'name': '朝阳', 'type': 'line'}
{'data': [137, 146, 154, 156, 176, 183, 171, 217, 239, 284, 288, 397, 395], 'name': '海淀', 'type': 'line'}
{'data': [58, 54, 74, 57, 82, 84, 93, 79, 114, 113, 133, 151, 201], 'name': '通州', 'type': 'line'}


In [53]:
#进行数据可视化
options = {
    'chart'   : {'zoomType':'xy'},
    'title'   : {'text': '发帖量统计'},
    'subtitle': {'text': '可视化统计图表'},
    'xAxis'   : {'categories': [i for i in get_all_dates('2015.12.24', '2016.01.05')]},
    'yAxis'   : {'title': {'text': '数量'}}
    }

series = [i for i in get_data_within('2015.12.24', '2016.01.05', ['朝阳', '海淀', '通州'])]

charts.plot(series, options=options,show='inline')
# options=dict(title=dict(text='Charts are AWESOME!!!'))